# Playing with model editing

In [1]:
from pathlib import Path
if Path().resolve().name == "notebooks":
    %cd ..
%ls
import sys
sys.path.append("models")
sys.path.append("data")
sys.path.append("concepts")

/Users/dgcnz/development/uva/fact/FACT
LICENSE                               models/
README.md                             notebook/
accuracies.csv                        notebooks/
accuracies2.csv                       old_logs/
artifacts/                            poetry.lock
assets/                               pyproject.toml
concepts/                             scripts/
configs/                              slurm/
data/                                 test_cav_activation.py
debug.log                             test_different_projections.py
docs/                                 testnotebook.ipynb
environment.yaml                      train_pcbm.py
evaluate_og_model.py                  train_pcbm_h.py
experiments/                          train_pcbm_userstudy.py
extension_scripts/                    training_tools/
learn_concepts_dataset.py             verify_clip_pcbm_h.py
learn_concepts_multimodal.py          verify_dataset_pcbm.py
lightning_logs/                       verify_d

In [2]:
from models import PosthocLinearCBM, get_model
from collections import namedtuple
from data import get_dataset
from concepts import ConceptBank
import pickle
import torch

In [25]:
BACKBONE_NAME = "clip:RN50"
DEVICE = "cpu"
CONCEPT_BANK = "artifacts/outdir/broden_clip_RN50_10.0_50.pkl"
PCBM_PATH = "artifacts/outdir/pcbm_cifar10__clip_RN50__broden_clip_RN50_10__lam_0.0002__alpha_0.99__seed_42.ckpt"
args = namedtuple(
    "Args",
    [
        "out_dir",
        "device",
        "dataset",
        "batch_size",
        "num_workers",
    ],
)(
    out_dir="artifacts/outdir",
    device=DEVICE,
    dataset="cifar10",
    batch_size=64,
    num_workers=4,
)
all_concepts = pickle.load(open(CONCEPT_BANK, 'rb'))
all_concept_names = list(all_concepts.keys())
print(f"Bank path: {CONCEPT_BANK}. {len(all_concept_names)} concepts will be used.")
concept_bank = ConceptBank(all_concepts, args.device)
backbone, preprocess = get_model(args, backbone_name=BACKBONE_NAME)
train_loader, test_loader, idx_to_class, classes = get_dataset(args, preprocess)
num_classes = len(classes)
backbone = backbone.to(DEVICE)
backbone.eval()
# posthoc_layer = PosthocLinearCBM(
#     concept_bank,
#     backbone_name=BACKBONE_NAME,
#     idx_to_class=idx_to_class,
#     n_classes=num_classes,
# )
# posthoc_layer = posthoc_layer.to(args.device)
posthoc_layer = torch.load(PCBM_PATH, map_location=torch.device('cpu'))

Bank path: artifacts/outdir/broden_clip_RN50_10.0_50.pkl. 170 concepts will be used.
Concept Bank is initialized.
Files already downloaded and verified
Files already downloaded and verified


In [16]:
print(len(concept_bank.concept_names))

170


In [28]:
concept_bank.concept_info.concept_names.index("dog")

86

In [ ]:
concept_bank

In [17]:
d = dict(zip(concept_bank.concept_names, list(range(len(concept_bank.concept_names)))))
d["dog"]

86

In [19]:
d["cat"]

128

In [15]:
list(d.keys())[:10]

['door_frame',
 'concrete',
 'horse',
 'food',
 'oven',
 'mouse',
 'bedroom_s',
 'book',
 'chain_wheel',
 'blueness']

In [11]:
idx_to_class

{0: 'airplane',
 1: 'automobile',
 2: 'bird',
 3: 'cat',
 4: 'deer',
 5: 'dog',
 6: 'frog',
 7: 'horse',
 8: 'ship',
 9: 'truck'}

In [10]:
all_concepts["door_frame"][0].shape

(1, 1024)

In [ ]:
all_concepts[""]

In [47]:
posthoc_layer.eval()

PosthocLinearCBM(
  (classifier): Linear(in_features=170, out_features=10, bias=True)
)

In [53]:
posthoc_layer.classifier.weight.shape

torch.Size([10, 170])

In [49]:
posthoc_layer.analyze_classifier().split("\n")

['Class : airplane',
 '\t 1 - airplane: 13.058',
 '\t 2 - body: 5.333',
 '\t 3 - bench: 5.237',
 '\t 4 - fluorescent: 5.024',
 '\t 5 - building: 4.217',
 'Class : automobile',
 '\t 1 - headlight: 13.475',
 '\t 2 - car: 7.165',
 '\t 3 - door: 4.991',
 '\t 4 - flower: 3.321',
 '\t 5 - motorbike: 3.272',
 'Class : bird',
 '\t 1 - bird: 9.885',
 '\t 2 - house: 7.090',
 '\t 3 - foot: 6.455',
 '\t 4 - ashcan: 4.173',
 '\t 5 - handle_bar: 3.579',
 'Class : cat',
 '\t 1 - cat: 9.937',
 '\t 2 - foot: 6.573',
 '\t 3 - floor: 6.013',
 '\t 4 - street_s: 4.118',
 '\t 5 - ceramic: 3.946',
 'Class : deer',
 '\t 1 - street_s: 8.018',
 '\t 2 - tree: 6.953',
 '\t 3 - engine: 6.178',
 '\t 4 - clock: 5.866',
 '\t 5 - pipe: 5.183',
 'Class : dog',
 '\t 1 - paw: 8.105',
 '\t 2 - muzzle: 6.343',
 '\t 3 - board: 6.019',
 '\t 4 - exhaust_hood: 5.522',
 '\t 5 - keyboard: 5.193',
 'Class : frog',
 '\t 1 - hand: 6.303',
 '\t 2 - greenness: 6.221',
 '\t 3 - food: 4.601',
 '\t 4 - carpet: 4.462',
 '\t 5 - pipe: 4.1

In [3]:
from models.pcbm_pl import PCBMClassifierTrainer
from lightning import Trainer
from data.metashift import MetaShiftDataModule, PreprocessorEnum
from data.metashift import NNProjector
from models.clip_encoder import CLIPImageEncoder
from lightning.pytorch import seed_everything

In [4]:
ckpt_path = "lightning_logs/task_1_bed_cat_dog/seed_42/version_0/checkpoints/epoch=19-step=1260.ckpt"
pcbm = PCBMClassifierTrainer.load_from_checkpoint(ckpt_path)

In [4]:
metashift = MetaShiftDataModule(
    task_name="task_1_bed_cat_dog",
    projector=NNProjector(
        concept_bank_path="artifacts/outdir/broden_clip_RN50_10.0_50.pkl",
        backbone=CLIPImageEncoder(model_name="RN50"),
    ),
    preprocessor_name=PreprocessorEnum.CLIP_RESNET50,
    train_batch_size=16,
    test_batch_size=64
)

Concept Bank is initialized.


In [14]:
metashift.dataset["train"].info.features["label"]

ClassLabel(names=['airplane', 'bed', 'car', 'cow', 'keyboard'], id=None)

In [6]:
trainer = Trainer(deterministic=True)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [7]:
seed_everything(42)
trainer.test(pcbm, metashift)

Seed set to 42


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/Users/dgcnz/development/uva/fact/FACT/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9139999747276306     │
└───────────────────────────┴───────────────────────────┘

[{'test_accuracy': 0.9139999747276306}]

In [8]:
#   pruned_concept_class_pairs:
#     - [128, 1]
pcbm_pruned = PCBMClassifierTrainer.load_from_checkpoint(ckpt_path, pruned_concept_class_pairs=[(128, 1)])

In [24]:
pcbm_pruned.model.classifier.weight[1]

tensor([-1.8379e-01, -1.4750e+00, -7.9499e-01, -8.7337e-01,  4.6112e-01,
        -1.2257e+00, -1.1098e+00,  2.7706e+00, -2.7329e-01, -6.3556e-01,
         8.7478e-01,  4.0155e-01,  5.9591e-02,  2.5387e-01, -5.3535e-01,
         1.2156e+00, -3.9201e-01,  1.1857e+00,  1.4236e+00,  3.4622e+00,
         1.0951e-01, -1.3728e-01, -2.7274e-01, -6.8375e-02, -2.1637e+00,
        -6.9077e-01,  3.4282e+00,  1.8606e+00, -9.9328e-01, -1.3042e+00,
        -1.8206e+00, -3.2627e-03, -1.1488e+00,  2.7601e+00, -3.9218e+00,
         1.4030e-01,  1.6988e+00, -3.8362e-01,  1.7959e+00, -1.2532e+00,
        -4.5289e-01, -6.5062e-01,  1.1795e-01, -1.7453e+00, -6.2271e-01,
         1.7091e+00, -1.3507e+00,  2.0358e+00, -1.7186e+00,  1.2518e+00,
        -1.0837e+00, -2.4383e+00,  4.6233e-01,  2.5421e-01,  1.6956e+00,
         2.0141e+00, -1.7333e+00, -2.3271e+00, -1.6009e+00, -1.8261e+00,
        -9.5668e-01, -1.9399e+00, -1.3950e+00,  1.3293e+00,  1.0415e+00,
        -3.6348e-01,  1.1920e+00,  9.7176e-01,  1.1

In [23]:
pcbm.model.classifier.weight[1]

tensor([-1.8379e-01, -1.4750e+00, -7.9499e-01, -8.7337e-01,  4.6112e-01,
        -1.2257e+00, -1.1098e+00,  2.7706e+00, -2.7329e-01, -6.3556e-01,
         8.7478e-01,  4.0155e-01,  5.9591e-02,  2.5387e-01, -5.3535e-01,
         1.2156e+00, -3.9201e-01,  1.1857e+00,  1.4236e+00,  3.4622e+00,
         1.0951e-01, -1.3728e-01, -2.7274e-01, -6.8375e-02, -2.1637e+00,
        -6.9077e-01,  3.4282e+00,  1.8606e+00, -9.9328e-01, -1.3042e+00,
        -1.8206e+00, -3.2627e-03, -1.1488e+00,  2.7601e+00, -3.9218e+00,
         1.4030e-01,  1.6988e+00, -3.8362e-01,  1.7959e+00, -1.2532e+00,
        -4.5289e-01, -6.5062e-01,  1.1795e-01, -1.7453e+00, -6.2271e-01,
         1.7091e+00, -1.3507e+00,  2.0358e+00, -1.7186e+00,  1.2518e+00,
        -1.0837e+00, -2.4383e+00,  4.6233e-01,  2.5421e-01,  1.6956e+00,
         2.0141e+00, -1.7333e+00, -2.3271e+00, -1.6009e+00, -1.8261e+00,
        -9.5668e-01, -1.9399e+00, -1.3950e+00,  1.3293e+00,  1.0415e+00,
        -3.6348e-01,  1.1920e+00,  9.7176e-01,  1.1

In [9]:
pcbm.model.classifier.weight[1][128]

tensor(1.9985, grad_fn=<SelectBackward0>)

In [10]:
pcbm_pruned.model.classifier.weight[1][128]

tensor(0., device='mps:0', grad_fn=<SelectBackward0>)

In [11]:
seed_everything(42)
trainer.test(pcbm_pruned, metashift)

Seed set to 42


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9139999747276306     │
└───────────────────────────┴───────────────────────────┘

[{'test_accuracy': 0.9139999747276306}]

In [8]:
pcbm_trainer = PCBMClassifierTrainer(n_concepts=170, n_classes=5, lr=0.05, lam=0.05, alpha=0.99)


In [10]:
seed_everything(42)
trainer = Trainer(deterministic=True, max_epochs=30)

Seed set to 42
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [60]:
seed = 100

In [61]:
seed_everything(seed)
trainer.fit(pcbm_trainer, metashift)

Seed set to 100

  | Name  | Type           | Params
-----------------------------------------
0 | model | PCBMClassifier | 855   
1 | loss  | PCBMLoss       | 855   
-----------------------------------------
855       Trainable params
0         Non-trainable params
855       Total params
0.003     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=30` reached.


In [62]:
seed_everything(seed)
out = trainer.test(pcbm_trainer, metashift)

Seed set to 100
/Users/dgcnz/development/uva/fact/FACT/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.8939999938011169     │
└───────────────────────────┴───────────────────────────┘

In [63]:
print(out)

[{'test_accuracy': 0.8939999938011169}]


In [64]:
trainer.checkpoint_callback.best_model_path

'/Users/dgcnz/development/uva/fact/FACT/lightning_logs/version_1/checkpoints/epoch=29-step=960.ckpt'

In [65]:
pcbm_trainer_pruned = PCBMClassifierTrainer.load_from_checkpoint(trainer.checkpoint_callback.best_model_path,
                                                          pruned_concept_class_pairs=[(128, 1)]
                                                         )

In [66]:
pcbm_trainer_pruned.model.classifier.weight[1, 128]

tensor(0., device='mps:0', grad_fn=<SelectBackward0>)

In [67]:
pcbm_trainer.model.classifier.weight[1, 128]

tensor(1.2930, grad_fn=<SelectBackward0>)

In [68]:
seed_everything(seed)
pruned_out = trainer.test(pcbm_trainer_pruned, metashift)

Seed set to 100


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.8980000019073486     │
└───────────────────────────┴───────────────────────────┘

In [69]:
pruned_out

[{'test_accuracy': 0.8980000019073486}]

In [70]:
pruned_out[-1]["test_accuracy"] - out[-1]["test_accuracy"]

0.0040000081062316895

In [16]:
path = Path("lightning_logs/task_1_bed_dog_cat/seed_42/version_6/checkpoints/epoch=19-step=640.ckpt")
print(path)
pcbm = PCBMClassifierTrainer.load_from_checkpoint(str(path))

lightning_logs/task_1_bed_dog_cat/seed_42/version_6/checkpoints/epoch=19-step=640.ckpt


In [4]:
from models.pcbm_pl import analyze_classifier
from models.resnet import ResNet18FeatureExtractor

In [8]:
CONCEPT_BANK = "artifacts/outdir/broden_resnet18_imagenet1k_v1_0.1_50.pkl"
all_concepts = pickle.load(open(CONCEPT_BANK, 'rb'))
all_concept_names = list(all_concepts.keys())
concept_bank = ConceptBank(all_concepts, args.device)

Concept Bank is initialized.


In [11]:
metashift_resnet18 = MetaShiftDataModule(
    task_name="task_1_bed_dog_cat",
    projector=NNProjector(
        concept_bank_path="artifacts/outdir/broden_resnet18_imagenet1k_v1_0.1_50.pkl",
        backbone=ResNet18FeatureExtractor(),
    ),
    preprocessor_name=PreprocessorEnum.RESNET18_IMAGENET_1K_V1,
    train_batch_size=16,
    test_batch_size=64
)
metashift_resnet18.setup("test")

Concept Bank is initialized.


In [12]:
metashift_resnet50 = MetaShiftDataModule(
    task_name="task_1_bed_dog_cat",
    projector=NNProjector(
        concept_bank_path="artifacts/outdir/broden_clip_RN50_10.0_50.pkl",
        backbone=CLIPImageEncoder(model_name="RN50"),
    ),
    preprocessor_name=PreprocessorEnum.CLIP_RESNET50,
    train_batch_size=16,
    test_batch_size=64
)
metashift_resnet50.setup("test")

Concept Bank is initialized.


In [8]:
import pickle
with open("artifacts/outdir/broden_clip_RN50_10.0_50.pkl", "rb") as f:
    concept_dict = pickle.load(f)

dict

In [14]:
cons = ["cat", "dog", "book"]
for con in cons:
    print(metashift_resnet18.projector.concept_names.index(con))

50
72
30


In [15]:
for con in cons:
    print(metashift_resnet50.projector.concept_names.index(con))

50
72
30


In [18]:
pcbm.model.classifier.weight[1, 72]

tensor(-0.3531, device='mps:0', grad_fn=<SelectBackward0>)

In [17]:
s =analyze_classifier(
    pcbm.model,
    concept_names=metashift_resnet18.projector.concept_names,
    class_names=metashift_resnet18.dataset["train"].info.features["label"].names,
)
print(s)

Class : airplane
	 1 - jar: 2.947
	 2 - pillow: 1.865
	 3 - ceramic: 1.854
	 4 - hair: 1.734
	 5 - paper: 1.538
Class : bed
	 1 - mouse: 2.838
	 2 - clock: 2.642
	 3 - bus: 2.136
	 4 - balcony: 1.924
	 5 - bicycle: 1.461
Class : car
	 1 - handle: 1.552
	 2 - chandelier: 1.310
	 3 - headlight: 1.309
	 4 - bench: 1.294
	 5 - fireplace: 1.251
Class : cow
	 1 - redness: 1.824
	 2 - field: 1.820
	 3 - bird: 1.780
	 4 - bedclothes: 1.732
	 5 - minibike: 1.701
Class : keyboard
	 1 - sofa: 2.877
	 2 - laminate: 2.399
	 3 - bicycle: 2.219
	 4 - blotchy: 1.948
	 5 - air_conditioner: 1.788


In [10]:
concept_bank.concept_info.concept_names

['cat',
 'paper',
 'apron',
 'microwave',
 'tree',
 'pane',
 'lamp',
 'polka_dots',
 'flowerpot',
 'fluorescent',
 'hand',
 'drawer',
 'fireplace',
 'figurine',
 'blind',
 'bench',
 'basket',
 'footboard',
 'chandelier',
 'handle_bar',
 'flag',
 'bedclothes',
 'drinking_glass',
 'car',
 'chain_wheel',
 'can',
 'balcony',
 'eye',
 'bed',
 'pipe',
 'box',
 'cup',
 'manhole',
 'granite',
 'desk',
 'back',
 'greenness',
 'cushion',
 'ceramic',
 'water',
 'dog',
 'plant',
 'board',
 'handle',
 'metal',
 'foot',
 'concrete',
 'armchair',
 'curtain',
 'ground',
 'person',
 'pedestal',
 'grass',
 'stripes',
 'flower',
 'bridge',
 'ottoman',
 'blotchy',
 'redness',
 'coffee_table',
 'path',
 'cardboard',
 'cabinet',
 'laminate',
 'jar',
 'body',
 'outside_arm',
 'oven',
 'pack',
 'chair',
 'exhaust_hood',
 'napkin',
 'paw',
 'bus',
 'chest_of_drawers',
 'hill',
 'plate',
 'frame',
 'nose',
 'mouth',
 'column',
 'doorframe',
 'headlight',
 'loudspeaker',
 'motorbike',
 'inside_arm',
 'pillar',
 

In [ ]:
analyze_classifier(pcbm.model, class_names=)